In [5]:
from nibabel.freesurfer.io import read_morph_data, write_morph_data # allows to read and write those curv files
import pickle
# read in pRF-estimates file
# for prf analysis
%matplotlib inline
from linescanning import prf, plotting, fitting
import numpy as np
import os
from scipy import io
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
opd = os.path.dirname
opj = os.path.join

In [7]:
subnum = '007'
subject = 'sub-'+subnum


## Get placebo session 
if subnum in ['003', '004', '007', '012', '016']: # placebo session 3
        
        ses = 'ses-3'
                
else:  ## placebo session 2   
        
        ses = 'ses-2'

## Fit the Gaussian model to the preprocessed median BOLD signal

In [ ]:
timecourse = np.load(f'/data1/projects/Meman1/projects/pilot/derivatives/prf/{subject}/{ses}/{subject}_{ses}_task-2R_hemi-LR_desc-avg_bold.npy') 

designmatrix = prf.read_par_file(opj(opd(opd(prf.__file__)), '/data1/projects/Meman1/projects/pilot/code', 'design_task-2R.mat'))

In [ ]:
### the parameters optimization is perfromed in the linux console, across several jobs in order to reduce time and CPU

# we initiate the model as per usual
g_load = prf.pRFmodelFitting(
    timecourse.T,
    design_matrix=designmatrix,
    TR=1.5,
    verbose=True,
    model = 'gauss',
    stage = 'iter',
    constraints= 'bgfs')

**The parameters optimization is perfromed in the linux console, across several jobs in order to reduce time and CPU**

In [8]:
### ses-2 ###
prf_file = f'/data1/projects/Meman1/projects/pilot/derivatives/prf/{subject}/{ses}/{subject}_{ses}_task-2R_model-gauss_stage-iter_desc-prf_params.pkl'

prf_results = prf.read_par_file(prf_file)
print(prf_results.shape)

(339716, 6)


## Calculate polarity and eccentricity

In [9]:
# actually extract the prf measurements:
eccentricity = np.sqrt( prf_results[:,0]**2 + prf_results[:,1]**2 ) # x and y squared

polar_angle = np.arctan2( prf_results[:,1], prf_results[:,0] )

r2 = prf_results[:,-1] #r2 values

print(eccentricity, polar_angle, r2)

[6.13348544 5.53093423 5.39789653 ... 0.         0.         0.        ] [-1.01113632 -1.50138396 -1.47654575 ...  0.          0.
  0.        ] [0.44598526 0.16080613 0.25178902 ... 0.         0.         0.        ]


In [10]:
# some code that illustrates the masking:
r2_mask = r2 > 0.1 #“float with your r squared threshold” # creates Boolean array with those vertices above the threshold

#ecc_mask = eccentricity < np.max(eccentricity)# “float with your max eccentricity value” / it should be like 5 radius
ecc_mask = eccentricity < 5 # (visual space radius) it should be like 5 radius

total_mask = r2_mask * ecc_mask # Boolean array where both above r squared and below maximum eccentricity
# note: you could include different things into your mask, such as minimal signal intensity, minimum eccentricity etc. 
 
#masked_polar_angle = np.where(total_mask, polar_angle, -10) # take only polar
masked_polar_angle = np.where(r2_mask, polar_angle, -10) # take only polar

#angle values for those vertices we want to include, put -10 for all others (so we can exclude them for plotting)
masked_eccentricity = np.where(total_mask, eccentricity, -10)

# freeview has separate curv files for left and right hemisphere, so if your prf analysis results are combined hemispheres we need to split them first:
print(np.sum(r2_mask)) # 27880 / 327099

print(np.sum(ecc_mask)) #288670 / 327099

print(np.sum(total_mask)) #18160 / 327099

39234
328553
28071


## Save eccentricity and polarity mask to plot in the inflated surface (Freesurfer)

In [11]:
lh_c = read_morph_data(f'/data1/projects/Meman1/projects/pilot/derivatives/freesurfer/{subject}/surf/lh.curv') # this is just to get the number of vertices for the left hemisphere. With that we can split the prf_results into the separate hemispheres. The curv files should be located in your freesurfer/subID/surfs folder.
 
lh_masked_pol = masked_polar_angle[:lh_c.shape[0]] # left hemisphere = all vertices up to the index
rh_masked_pol = masked_polar_angle[lh_c.shape[0]:] # all vertices from that vertex onwards belong to the right hemisphere

lh_masked_ecc = masked_eccentricity[:lh_c.shape[0]] # left hemisphere = all vertices up to the index
rh_masked_ecc = masked_eccentricity[lh_c.shape[0]:] # all vertices from that vertex onwards belong to the right hemisphere
 
# now save the results as curv file:
fs_dir = f'/data1/projects/Meman1/projects/pilot/derivatives/freesurfer/{subject}/surf'
 
write_morph_data(opj(fs_dir, 'lh.masked_pol'), lh_masked_pol) #opj(fs_dir, <name you want to give the new curv file, something like “lh.masked_pol”>), lh_masked_pol
write_morph_data(opj(fs_dir, 'rh.masked_pol'), rh_masked_pol)

write_morph_data(opj(fs_dir, 'lh.masked_ecc'), lh_masked_ecc) #opj(fs_dir, <name you want to give the new curv file, something like “lh.masked_pol”>), lh_masked_pol
write_morph_data(opj(fs_dir, 'rh.masked_ecc'), rh_masked_ecc)

## FreeSurfer

**Open freeview with the polar mask as an overlay layer on the inflated surface of the subject. Overlay_custom allows to color-encode the angles.**

*In the terminal , to open freeview (from the surface directory of the subjects):*

In [ ]:
cd /data1/projects/Meman1/projects/pilot/derivatives/freesurfer/{subject}/surf

*overlay_custom=-3.14,255,0,0,:* refers to -3.14 degrees and {255, 0, 0} is the rgb color for this polar angle. Thus, the following numbers refer to a combination of the angle and the 3 rgb values to encode it.


In [ ]:
freeview -f lh.inflated:overlay=lh.masked_pol:overlay_custom=-3.14,255,0,0,-2.65,255,255,0,-2.09,0,128,0,-1.75,0,255,255,-1.05,0,0,255,-0.5,238,130,238,0,255,0,0,0.5,255,255,0,1.05,0,128,0,1.57,0,255,255,2.09,0,0,255,2.65,238,130,238,3.14,255,0,0 rh.inflated:overlay=rh.masked_pol:overlay_custom=-3.14,255,0,0,-2.65,255,255,0,-2.09,0,128,0,-1.75,0,255,255,-1.05,0,0,255,-0.5,238,130,238,0,255,0,0,0.5,255,255,0,1.05,0,128,0,1.57,0,255,255,2.09,0,0,255,2.65,238,130,238,3.14,255,0,0

In [ ]:
freeview -f lh.inflated:overlay=lh.masked_ecc:overlay_custom=0,255,0,0,1,255,255,0,2,0,128,0,3,0,255,255,4,128,0,128 rh.inflated:overlay=rh.masked_ecc:overlay_custom=0,255,0,0,1,255,255,0,2,0,128,0,3,0,255,255,4,128,0,128